In [1]:
import pipeline_models as pl
import pandas as pd
import numpy as np
import os

from IPython.display import display, HTML
from IPython.display import clear_output

import time

# def set_random(seed_value):
#     # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
#     os.environ['PYTHONHASHSEED']=str(seed_value)
#     # 2. Set `python` built-in pseudo-random generator at a fixed value
#     random.seed(seed_value)
#     # 3. Set `numpy` pseudo-random generator at a fixed value
#     np.random.seed(seed_value)
#     # 4. Set `tensorflow` pseudo-random generator at a fixed value
#     tf.random.set_seed(seed_value)

# set_random(0)

In [2]:
# np.random.seed(0)
# np.random.choice(10, 3, replace=False).tolist()
# np.random.seed(0)
# np.random.get_state()[1][0]

In [3]:
all_files=[]
all_files_name = []
for root, dirs, files in os.walk("./data/"):
    for file in files:
        if file.endswith(".csv"):
            all_files.append(os.path.join(root, file))
            all_files_name.append(root.split("/")[-1] + "_" + file.replace(".", "_"))
# df = pd.read_csv()
for ind, tmp_model_name in enumerate(["LSTM", "LSTM", "LSTM", "AE"]):
    for i in range(1,30):
        df = pd.read_csv(all_files[i], 
                    sep=';', 
                    index_col='datetime', 
                    parse_dates=True)
        tmp_model_obj = None
        if tmp_model_name == "LSTM":
            tmp_model_obj = pl.PipelineModel(capture_info=True, infoWriter_args=["tmp_res", "family_LSTM", f"model_name_{ind}", f"data_name_{i}"])
        if tmp_model_name == "AE":
            tmp_model_obj = pl.PipelineModel(capture_info=True, infoWriter_args=["tmp_res", "family_AE", f"AE", f"data_name_{i}"])
        
        tmp_model_obj.apply_standard_scaling(())
        tmp_model_obj.apply_nested_rotations((2, 0.75))
        tmp_model_obj.apply_feature_bagging(())
        if tmp_model_name == "LSTM":
            tmp_model_obj.apply_model(("LSTM", f"model_name_{ind}"))
        if tmp_model_name == "AE":
            tmp_model_obj.apply_model(("AE", f"model_name_{ind}"))
        

        tmp_model_obj.pipeline_fit(df[:400].drop(['anomaly', 'changepoint'], axis=1))

        final_df = tmp_model_obj.pipeline_transform(df.drop(['anomaly', 'changepoint'], axis=1))

In [ ]:
df = pd.DataFrame([
    (1,2,["a1", "a2"], [1,2,3,4],[[1,2],[3,4,5]],[["asd1", "asd2"], ["1", "2", "3"]]),
    (1,2,["a1", "a2"], [1,2,3,4],[[1,2],[3,4,5]],[["asd1", "asd2"], ["1", "2", "3"]]),
    (1,2,["a1", "a2"], [1,2,3,4],[[1,2],[3,4,5]],[["asd1", "asd2"], ["1", "2", "3"]]),
], columns=["aa", "bb", "cc", "dd", "ee", "ff"], index = ["a1", "b1", "c1"])
# df["asd"] = df.apply(lambda x: type(x.ee), axis=1)
df

In [ ]:
df.to_csv("./abc.csv", header=True, index=False)
df

In [ ]:
df = pd.read_csv("./abc.csv", index_col=0)
df

In [ ]:
df.to_parquet("abc.parquet.gzip", compression='gzip')
df.to_parquet("abc.parquet", compression=None)
df.to_csv("./abc.csv", header=True, index=True)

In [ ]:
import os

# -------------------------------------------
def breadthFirstFileScan( root ) :
    dirs = [root]
    # while we has dirs to scan
    while len(dirs) :
        nextDirs = []
        for parent in dirs :
            # scan each dir
            for f in os.listdir( parent ) :
                # if there is a dir, then save for next ittr
                # if it  is a file then yield it (we'll return later)
                ff = os.path.join( parent, f )
                if os.path.isdir( ff ) :
                    nextDirs.append( ff )
                else :
                    yield ff
        # once we've done all the current dirs then
        # we set up the next itter as the child dirs 
        # from the current itter.
        dirs = nextDirs

# -------------------------------------------
# an example func that just outputs the files.
def walkbf( path ) :
    for f in breadthFirstFileScan( path ) :
        print f

# ============================================
# as a demo we'll just start from where we 
# were called from.
walkbf( os.getcwd() )

In [27]:
print(os.listdir("./tmp_res/"))
for family in os.listdir("./tmp_res/"):
    print(f'\t{os.path.join("./tmp_res/", family)}->{os.listdir(os.path.join("./tmp_res/", family))}')
    for model_name in os.listdir(f"./tmp_res/{family}"):
        print(f'\t\t{os.listdir(os.path.join("./tmp_res/",family, model_name))}')

['family_AE', 'family_LSTM', 'lstm_family']
	./tmp_res/family_AE->['AE']
		['data_name_1', 'data_name_10', 'data_name_11', 'data_name_12', 'data_name_13', 'data_name_14', 'data_name_15', 'data_name_16', 'data_name_17', 'data_name_18', 'data_name_19', 'data_name_2', 'data_name_20', 'data_name_21', 'data_name_22', 'data_name_23', 'data_name_24', 'data_name_25', 'data_name_26', 'data_name_27', 'data_name_28', 'data_name_29', 'data_name_3', 'data_name_4', 'data_name_5', 'data_name_6', 'data_name_7', 'data_name_8', 'data_name_9']
	./tmp_res/family_LSTM->['model_name_0', 'model_name_1', 'model_name_2']
		['data_name_1', 'data_name_10', 'data_name_11', 'data_name_12', 'data_name_13', 'data_name_14', 'data_name_15', 'data_name_16', 'data_name_17', 'data_name_18', 'data_name_19', 'data_name_2', 'data_name_20', 'data_name_21', 'data_name_22', 'data_name_23', 'data_name_24', 'data_name_25', 'data_name_26', 'data_name_27', 'data_name_28', 'data_name_29', 'data_name_3', 'data_name_4', 'data_name_5'

In [13]:
list(os.walk("./tmp_res/"))

[('./tmp_res/', ['family_AE', 'family_LSTM', 'lstm_family'], []),
 ('./tmp_res/family_AE', ['AE'], []),
 ('./tmp_res/family_AE\\AE',
  ['data_name_1',
   'data_name_10',
   'data_name_11',
   'data_name_12',
   'data_name_13',
   'data_name_14',
   'data_name_15',
   'data_name_16',
   'data_name_17',
   'data_name_18',
   'data_name_19',
   'data_name_2',
   'data_name_20',
   'data_name_21',
   'data_name_22',
   'data_name_23',
   'data_name_24',
   'data_name_25',
   'data_name_26',
   'data_name_27',
   'data_name_28',
   'data_name_29',
   'data_name_3',
   'data_name_4',
   'data_name_5',
   'data_name_6',
   'data_name_7',
   'data_name_8',
   'data_name_9'],
  []),
 ('./tmp_res/family_AE\\AE\\data_name_1',
  [],
  ['nested_rotations.parquet.gzip',
   'other_info.parquet.gzip',
   'partition_0.parquet.gzip',
   'partition_1.parquet.gzip',
   'scalings.parquet.gzip',
   'scores_and_anomalies.parquet.gzip']),
 ('./tmp_res/family_AE\\AE\\data_name_10',
  [],
  ['nested_rotations.p

In [10]:
all_files=[]
all_files_name = []
for root, dirs, files in os.walk("./tmp_res/"):
    print(f"root={root}")
    print(f"dirs={dirs}")
    print(f"file={files}")
    print("===============")
    # for file in files:
    # print(f"root={root}")
    # print(f"dirs={dirs}")
    # print(f"file={file}")
    # print("===============")

root=./tmp_res/
dirs=['family_AE', 'family_LSTM', 'lstm_family']
file=[]
root=./tmp_res/family_AE
dirs=['AE']
file=[]
root=./tmp_res/family_AE\AE
dirs=['data_name_1', 'data_name_10', 'data_name_11', 'data_name_12', 'data_name_13', 'data_name_14', 'data_name_15', 'data_name_16', 'data_name_17', 'data_name_18', 'data_name_19', 'data_name_2', 'data_name_20', 'data_name_21', 'data_name_22', 'data_name_23', 'data_name_24', 'data_name_25', 'data_name_26', 'data_name_27', 'data_name_28', 'data_name_29', 'data_name_3', 'data_name_4', 'data_name_5', 'data_name_6', 'data_name_7', 'data_name_8', 'data_name_9']
file=[]
root=./tmp_res/family_AE\AE\data_name_1
dirs=[]
file=['nested_rotations.parquet.gzip', 'other_info.parquet.gzip', 'partition_0.parquet.gzip', 'partition_1.parquet.gzip', 'scalings.parquet.gzip', 'scores_and_anomalies.parquet.gzip']
root=./tmp_res/family_AE\AE\data_name_10
dirs=[]
file=['nested_rotations.parquet.gzip', 'other_info.parquet.gzip', 'partition_0.parquet.gzip', 'partition

In [ ]:
df = pd.read_parquet("./tmp_res/lstm_family/lstm_model/dataset1/other_info.parquet.gzip")
df["asd"] = df.apply(lambda x: 1 if x.info_value is None else x.info_value[0], axis=1)
display(df)


In [ ]:
df = pd.read_parquet("./abc.parquet.gzip")
df["asd"] = df.apply(lambda x : x.dd.tolist(),axis=1)
df["asdeq"] = df["asd"] == df["dd"]
df

In [ ]:
df = pd.read_csv("./abc.csv")
df["asd"] = df.apply(lambda x: type(x.dd), axis=1)
df

In [ ]:
                list_of_times                   total_times K franction  fbsubsets       actions        
Score_alg_1   [time1, time2, time3,....,time35]   12345     2    0.75    [1,2,3,8] ["fb", "Sc", "Rot"]    
Score_alg_2   [time1, time2, time3,....,time35]   12345     2    0.75    [1,2,4,8]
Score_alg_3   [time1, time2, time3,....,time35]   12345     2    0.75    [1,5,6,8]
Score_alg_4
Score_alg_5

In [ ]:
    Score_alg_1, UCL_alg_1, Score_alg_2, UCL_alg_2, Score_alg_3, UCL_alg_3
1
2
3
4
5
6

In [ ]:
start_time = time.perf_counter()
start_time_ns = time.perf_counter_ns()

In [ ]:
stop_time = time.perf_counter()
stop_time_ns = time.perf_counter_ns()

In [ ]:
stop_time - start_time

In [ ]:
(stop_time_ns - start_time_ns)/1000000000

In [ ]:
all_files=[]
all_files_name = []
for root, dirs, files in os.walk("./data/"):
    for file in files:
        if file.endswith(".csv"):
            all_files.append(os.path.join(root, file))
            all_files_name.append(root.split("/")[-1] + "_" + file.replace(".", "_"))
# df = pd.read_csv()
for model_name in ["LSTM", "LSTM", "LSTM", "AE"]:
    for i in range(1,30):
        df = pd.read_csv(all_files[i], 
                    sep=';', 
                    index_col='datetime', 
                    parse_dates=True)

        autoencoder = pl.PipelineModel()
        autoencoder.apply_standard_scaling(())
        autoencoder.apply_nested_rotations((2, 0.75))
        autoencoder.apply_feature_bagging(())
        autoencoder.apply_model(("CONV_AE", "task_name_1"))

        autoencoder.pipeline_fit(df[:400].drop(['anomaly', 'changepoint'], axis=1))

        final_df = autoencoder.pipeline_transform(df.drop(['anomaly', 'changepoint'], axis=1))

        final_df["final_anomaly"] = (final_df["anomaly_by_conv_ae_task_name_1_score"] > (3/2) * autoencoder.model.UCL).astype(int)
        print(f'MAX: {final_df["anomaly_by_conv_ae_task_name_1_score"].max()}, {3/2*autoencoder.model.UCL}, {(3/2)*autoencoder.model.UCL}')
        print(autoencoder.model.model_name)
        print(autoencoder.model.model)
        print(autoencoder.model.UCL)
        print(autoencoder.model.anomaly_col)
        display(final_df.groupby(["final_anomaly"]).count())

In [ ]:
import scaling_utils
import rotation_utils
import bagging_utils

In [ ]:
all_files=[]
all_files_name = []
for root, dirs, files in os.walk("./data/"):
    for file in files:
        if file.endswith(".csv"):
            all_files.append(os.path.join(root, file))
            all_files_name.append(root.split("/")[-1] + "_" + file.replace(".", "_"))
df = pd.read_csv(all_files[1], 
                sep=';', 
                index_col='datetime', 
                parse_dates=True)

StSc = scaling_utils.STDScaler()
StSc.fit(df[:400].drop(['anomaly', 'changepoint'], axis=1))
aaa = StSc.transform(df[:400].drop(['anomaly', 'changepoint'], axis=1))
FBag = bagging_utils.FeatureBagger()
FBag.fit(aaa)
aaa = FBag.transform(aaa)
Rot = rotation_utils.Rotator()
Rot.fit(aaa)
aaa = Rot.transform(aaa)
aaa

In [ ]:
print(type(StSc.StSc.scale_.tolist()))
print(type(StSc.StSc.mean_.tolist()))
print(type(StSc.StSc.var_.tolist()))
list(zip(StSc.StSc.scale_,StSc.StSc.mean_, StSc.StSc.var_))

In [ ]:
all_files=[]
all_files_name = []
for root, dirs, files in os.walk("./data/"):
    for file in files:
        if file.endswith(".csv"):
            all_files.append(os.path.join(root, file))
            all_files_name.append(root.split("/")[-1] + "_" + file.replace(".", "_"))
df = pd.read_csv(all_files[1], 
                sep=';', 
                index_col='datetime', 
                parse_dates=True)
print(df.shape)
autoencoder = pl.PipelineModel()
autoencoder.apply_standard_scaling(())
autoencoder.apply_nested_rotations((2, 0.75))
autoencoder.apply_feature_bagging(())

autoencoder.apply_model(("CONV_AE", "task_name_1"))


autoencoder.pipeline_fit(df[:400].drop(['anomaly', 'changepoint'], axis=1))
a = autoencoder.pipeline_transform(df.drop(['anomaly', 'changepoint'], axis=1))
print(f"{(3/2) * autoencoder.model.UCL} - {a['anomaly_by_conv_ae_task_name_1_score'].max()}")
a["final_anomaly"] = (a['anomaly_by_conv_ae_task_name_1_score'] > (3/2)*autoencoder.model.UCL).astype(int)
print(a.shape)
display(a.groupby(['final_anomaly']).count())
display(a)

In [ ]:
df.groupby(["anomaly"]).count()